In [3]:
# Import dependencies
import cv2
import numpy as np
import matplotlib.pyplot as plt

'''
Create a live video preview of the model using user's default camera.

    Parameters:
    -----------
    model : tf.keras.model
        CNN model used to predict emotion class

    Returns:
    --------
    None, just the live video stream

'''
# Capture and process video from webcam
def live_feed(model):
    
    # Define emotion labels
    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    
    # Setup video capture device
    cap = cv2.VideoCapture(0)  # 0 is the default camera
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            continue

        # Detect faces in the image
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Draw rectangles around detected faces and process the first detected face
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            face_img = gray_frame[y:y+h, x:x+w]
            resized_face = cv2.resize(face_img, (48, 48))

            # Preprocess the image for the model
            normalized_face = resized_face / 255.0  # Normalize pixel values
            input_face = normalized_face.reshape(1, 48, 48, 1)  # Add batch dimension

            # Predict emotion
            predictions = model.predict(input_face)
            emotion_index = np.argmax(predictions)
            emotion = emotion_labels[emotion_index]

            # Display emotion label on the frame
            cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        # Display the frame with detected faces and emotion labels
        cv2.imshow('Video Feed', frame)

        # Break loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()